In [52]:
import json
import pandas as pd
import woodwork as ww
import tqdm
import evalml

ModuleNotFoundError: No module named 'evalml'

In [2]:
with open('data/runners_with_data.json', 'r') as fp:
    data = json.load(fp)

In [3]:
data[0]

{'name': 'Tura Abdiwak, Seifu',
 'country': 'ETH',
 'age_class': '20-24',
 'half_time': '01:02:29',
 'finish_time': '02:06:12',
 'details_url': 'https://results.chicagomarathon.com/2021/?content=detail&idp=LSMG963824DD5C',
 'bib': '3',
 'city_state': 'Iseo',
 'splits': {'start': {'time_of_day': '07:30:00', 'time': '00:00:00'},
  '5km': {'time_of_day': '07:44:43AM', 'time': '00:14:43'},
  '10km': {'time_of_day': '07:59:15AM', 'time': '00:29:15'},
  '15km': {'time_of_day': '08:14:21AM', 'time': '00:44:21'},
  '20km': {'time_of_day': '08:29:13AM', 'time': '00:59:13'},
  'half': {'time_of_day': '08:32:29AM', 'time': '01:02:29'},
  '25km': {'time_of_day': '08:44:42AM', 'time': '01:14:42'},
  '30km': {'time_of_day': '09:00:06AM', 'time': '01:30:06'},
  '35km': {'time_of_day': '09:15:01AM', 'time': '01:45:01'},
  '40km': {'time_of_day': '09:29:44AM', 'time': '01:59:44'},
  'finish': {'time_of_day': '09:36:12AM', 'time': '02:06:12'}}}

In [53]:
def time_to_seconds(time):
    hour, minutes, seconds = time.split(":")
    return int(hour)*60*60 + int(minutes)*60 + int(seconds)

def make_feature(runner):
    features = {
     "age_class": runner["age_class"],   
     "country": runner["country"]
    }
    
    split_times = []
    split_diffs = []
    for split, value in runner["splits"].items():
        if value["time"] == "–":
            split_time = None
        else:
            split_time = time_to_seconds(value["time"])
            
        split_times.append(split_time)
            
        features[split+"_elapsed"] = split_times[-1]

        # todo do something to hand half
        if len(split_times) >= 2 and split_times[-1] and split_times[-2]:
            split_diff = split_times[-1] - split_times[-2]
            features[split+"_diff"] = split_diff
            split_diffs.append(split_diff)
            features[split+"_std"] = pd.Series(split_diffs).std()
            
    return features

df = pd.DataFrame([make_feature(d) for d in tqdm.tqdm(data)])


 12%|████▍                                 | 1671/14201 [04:38<34:51,  5.99it/s]

 25%|█████████▎                           | 3571/14201 [00:10<00:27, 382.47it/s]


 52%|███████████████████▍                 | 7443/14201 [00:20<00:17, 382.41it/s]


 80%|████████████████████████████▋       | 11292/14201 [00:30<00:07, 392.69it/s]


100%|████████████████████████████████████| 14201/14201 [00:37<00:00, 376.49it/s]


In [54]:
df

,age_class,country,start_elapsed,5km_elapsed,10km_elapsed,10km_diff,10km_std,15km_elapsed,15km_diff,15km_std,...,30km_std,35km_elapsed,35km_diff,35km_std,40km_elapsed,40km_diff,40km_std,finish_elapsed,finish_diff,finish_std
0,20-24,ETH,0,883.0,1755.0,872.0,NaN,2661.0,906.0,24.041631,...,281.700846,6301.0,895.0,262.633458,7184.0,883.0,246.186304,7572,388.0,266.037957
1,35-39,USA,0,883.0,1765.0,882.0,NaN,2663.0,898.0,11.313708,...,282.684984,6302.0,895.0,263.500429,7193.0,891.0,247.427767,7595,402.0,264.929131
2,30-34,KEN,0,883.0,1757.0,874.0,NaN,2661.0,904.0,21.213203,...,281.655404,6301.0,895.0,262.592841,7205.0,904.0,247.420261,7611,406.0,264.518483
3,25-29,JPN,0,884.0,1756.0,872.0,NaN,2662.0,906.0,24.041631,...,282.060573,6330.0,923.0,265.304856,7308.0,978.0,255.600134,7730,422.0,270.728646
4,20-24,ETH,0,876.0,1755.0,879.0,NaN,2646.0,891.0,8.485281,...,280.909001,6359.0,951.0,266.884940,7336.0,977.0,256.403700,7779,443.0,268.862883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14196,25-29,USA,0,2248.0,5101.0,2853.0,NaN,8231.0,3130.0,195.868578,...,1051.470336,22745.0,3185.0,966.516154,25930.0,3185.0,899.416438,27328,1398.0,989.450226
14197,40-44,USA,0,2249.0,5101.0,2852.0,NaN,8231.0,3130.0,196.575685,...,1052.218941,22747.0,3185.0,967.187454,25931.0,3184.0,899.994087,27330,1399.0,989.734406
14198,70-74,USA,0,2925.0,5986.0,3061.0,NaN,9136.0,3150.0,62.932504,...,993.022339,22554.0,3364.0,939.516441,25900.0,3346.0,890.670283,27369,1469.0,955.408290
14199,55-59,MEX,0,2694.0,5858.0,3164.0,NaN,9055.0,3197.0,23.334524,...,983.228441,23203.0,3966.0,1007.158946,26886.0,3683.0,969.722788,28495,1609.0,1022.392510


In [55]:
def seconds_to_time(seconds):
    hours = int(seconds / (60*60))
    minutes = int((seconds - hours*(60*60)) / 60)
    secs = int(seconds - hours*(60*60) - minutes*60) 
    return "{hours}:{minutes}:{seconds}".format(hours=hours, minutes=minutes, seconds=secs)

In [56]:
df.groupby("age_class")["finish_elapsed"].mean().map(seconds_to_time)

age_class
19 and under      5:1:1
20-24            4:20:1
25-29            4:18:8
30-34           4:18:10
35-39           4:22:10
40-44           4:25:54
45-49           4:35:24
50-54           4:43:18
55-59            4:53:2
60-64            5:1:57
65-69            5:23:7
70-74           5:48:52
75-79           5:58:43
80+             6:23:29
Name: finish_elapsed, dtype: object

In [67]:
select = ["age_class", "country", "5km_elapsed", "10km_elapsed", "10km_diff", "15km_elapsed", "15km_diff", "20km_elapsed", "20km_diff", "25km_elapsed", "25km_diff", "finish_elapsed"]
df_select = df[select].fillna(pd.NA)
df_select.ww.init()

In [68]:
df_select.ww.mutual_information()

TypeError: 'NoneType' object is not callable